<div class="alert alert-block" style = "background-color: black">
    <p><b><font size="+4" color="orange">Data Aggregation in Pandas</font></b></p>
    <p><b><font size="+1" color="white">by Jubril Davies</font></b></p>
    </div>

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
plt.rcParams.update({'font.size':14}) #sets global font size

$$\begin{align} \text{This work focuses on grouping & aggregating data for analysis} \end{align}$$
---
Categorizing a dataset and applying an operation to each group is a critical part of data analysis workflow. Pandas provides a flexible groupby function that allows you to slice and dice and summarize datasets naturally.

In this work, we will learn how to:

* Split pandas object into pieces using one or more keys
* Compute group summary statistics
* Apply a varying set of functions to each column of a DataFrame
* Apply group transformations such as normalizations, scaling etc
* Compute pivot tables and cross-tabulations
* Perform Quantile analysis and other derived group analyses 

---
<div class= "alert alert-block" style="background-color: orange; border-color: black">
    <p><b><font size="+2" color="black">Groupby Mechanics</font></b></p>
    </div>
  
---

Group operations follow the **split-apply-combine** rule.

* In the first stage, data is into groups based on one or more keys provided.
* A function is then applied to each group producing a new value
* Results are then combined into a result object

> #### **Given the famous titanic dataset**

In [4]:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
dat = pd.read_csv(url)
dat = dat.drop(columns = ['Name'])
data = dat.sample(10,random_state=52)
data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
580,581,1,2,female,25.0,1,1,237789,30.0000,NaN,S
841,842,0,2,male,16.0,0,0,S.O./P.P. 3,10.5000,NaN,S
406,407,0,3,male,51.0,0,0,347064,7.7500,NaN,S
394,395,1,3,female,24.0,0,2,PP 9549,16.7000,G6,S
453,454,1,1,male,49.0,1,0,17453,89.1042,C92,C
671,672,0,1,male,31.0,1,0,F.C. 12750,52.0000,B71,S
257,258,1,1,female,30.0,0,0,110152,86.5000,B77,S
528,529,0,3,male,39.0,0,0,3101296,7.9250,NaN,S
433,434,0,3,male,17.0,0,0,STON/O 2. 3101274,7.1250,NaN,S
773,774,0,3,male,NaN,0,0,2674,7.2250,NaN,C


**Pandas Groupby does a concise aggregation using a dataframes specified key**

<div style="background-color: black; padding: 5px">
    <p><b><font size="+2" color="white">1. Basic Grouping & Aggregation</font></b></p>
    </div>

### **1.1. Grouping by a Single Column**
The groupby method is used to group data. Lets group the titanic dataset by sex column and calculate the average age and fare for each gender.

In [5]:
grouped = data.groupby('Sex')[['Age','Fare']].mean()
grouped

,Age,Fare
Sex,,
female,26.333333,44.400000
male,33.833333,25.947029


**It is useful to obtain the group size using the `groupby method`. By default, missing values in a group key are excluded from the result. This behavior can be disabled by passing dropna=False to groupby.**

> #### **Get the average age of the passengers in each cabin**

In [6]:
cabin_group = data.groupby(['Cabin'])['Age'].size()
cabin_group

Cabin
B71    1
B77    1
C92    1
G6     1
Name: Age, dtype: int64

In [7]:
cabin_group = data.groupby('Cabin',dropna=False)['Age'].size()
cabin_group

Cabin
B71    1
B77    1
C92    1
G6     1
NaN    6
Name: Age, dtype: int64

> #### **To compute the number of nonnull values in each group, we use the count**

In [8]:
data.groupby('Cabin')['Age'].count()

Cabin
B71    1
B77    1
C92    1
G6     1
Name: Age, dtype: int64

### **1.2. Iterating over Groups**
It is possible to iterate over the object returned by groupby generating a sequence of 2-tuples containing the group name along with the chunk of data.

In [9]:
for (Sex,Pclass), group in data.groupby(['Sex','Pclass']):
    print((Sex,Pclass))
    print(group)

('female', 1)
     PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch  Ticket  Fare  \
257          258         1       1  female  30.0      0      0  110152  86.5   

    Cabin Embarked  
257   B77        S  
('female', 2)
     PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch  Ticket  Fare  \
580          581         1       2  female  25.0      1      1  237789  30.0   

    Cabin Embarked  
580   NaN        S  
('female', 3)
     PassengerId  Survived  Pclass     Sex   Age  SibSp  Parch   Ticket  Fare  \
394          395         1       3  female  24.0      0      2  PP 9549  16.7   

    Cabin Embarked  
394    G6        S  
('male', 1)
     PassengerId  Survived  Pclass   Sex   Age  SibSp  Parch      Ticket  \
453          454         1       1  male  49.0      1      0       17453   
671          672         0       1  male  31.0      1      0  F.C. 12750   

        Fare Cabin Embarked  
453  89.1042   C92        C  
671  52.0000   B71        S  
('male', 2)
    

**It is possible to compute the dictionary of the data pieces as a one-liner**

In [10]:
pieces = {Sex: group for Sex,group in data.groupby("Sex")}
pieces['female']

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
580,581,1,2,female,25.0,1,1,237789,30.0,NaN,S
394,395,1,3,female,24.0,0,2,PP 9549,16.7,G6,S
257,258,1,1,female,30.0,0,0,110152,86.5,B77,S


### **1.3. Group by Columns or Multiple Columns**
Indexing a groupby object created from a dataframe with a column name or an array of column names creates the effect of subsetting by columns for aggregation.

df.groupby("key1")["data1"] and df.groupby("key1")[["data2"]] are conveiniences for:

df["data1"].groupby(df["key1"]) and df[["data2"]].groupby(df["key1"])

It is possible to group by multiple columns by passing a list of column names. 
Lets group by sex and passenger class (Pclass)

In [11]:
multi_grouped = data.groupby(['Sex','Pclass'])['Fare'].mean()
multi_grouped

Sex     Pclass
female  1         86.50000
        2         30.00000
        3         16.70000
male    1         70.55210
        2         10.50000
        3          7.50625
Name: Fare, dtype: float64

**This gives a series with hierarchical index with unique pairs of keys observed. We can therefore unstack this**

In [12]:
multi_grouped.unstack()

Pclass,1,2,3
Sex,,,
female,86.5000,30.0,16.70000
male,70.5521,10.5,7.50625


### **1.4. Grouping with Dictionary & Series**

The grouping information may also exist in the form of a dictionary as opposed to an array.

> #### **Given a dataframe of students**

In [13]:
students = pd.DataFrame(np.random.randn(5,5),columns = ['Abel','Bale','Chris','Dave','Emma'],
                        index=['crimson','gold','silver','jade','violet'])
students.loc[2:3,['Bale','Chris']] = np.nan #add a few NA values
students

,Abel,Bale,Chris,Dave,Emma
crimson,0.138019,1.002650,-1.350539,-0.212013,2.049286
gold,-0.344755,1.466474,0.512843,-0.626248,0.497178
silver,-1.385649,NaN,NaN,-1.099470,-1.833694
jade,0.677994,-1.047175,1.056322,0.126958,-0.835542
violet,-0.451262,-0.307875,0.114516,-1.106631,0.590975


> **Now assuming we have a group correspondence for the columns and would like to sum up columns by group**

In [14]:
mapping = {'Abel':'red','Bale':'red','Chris':'blue','Dave':'blue','Emma':'red','Frank':'orange'}
mapping

{'Abel': 'red',
 'Bale': 'red',
 'Chris': 'blue',
 'Dave': 'blue',
 'Emma': 'red',
 'Frank': 'orange'}

> **Now lets group**

In [15]:
by_column = students.groupby(mapping ,axis=1).sum()
by_column

,blue,red
crimson,-1.562552,3.189954
gold,-0.113405,1.618897
silver,-1.099470,-3.219343
jade,1.183280,-1.204723
violet,-0.992116,-0.168162


### **1.5. Grouping by Index Levels**

Another type of grouping that can be done using hierarchically indexed data is grouping by index levels. This is aggregating using one of the levels of an axis index. To do this the level number or name is passed using the level keyword.

In [16]:
columns = pd.MultiIndex.from_arrays([['UK','UK','UK','HK','HK'],[1,2,4,1,4]],names=['county','base'])
county_df = pd.DataFrame(np.random.randn(4,5),columns=columns)
county_df

county        UK                            HK          
base           1         2         4         1         4
0      -0.873218  1.183190  0.806581  1.513037 -1.273764
1      -0.570242  0.971918 -1.060369 -0.444680 -0.154907
2       0.513600 -1.243075 -0.300300  0.547417  0.444295
3       0.167789 -0.610117 -1.489661  0.704754  0.170270

<div style="background-color: black; padding: 5px">
    <p><b><font size="+2" color="white">2. Aggregation Functions</font></b></p>
    </div>

### **2.1. Grouping with built-in Aggregation Functions**

>#### **Given a dataframe of student scores**
You can supply a list of functions to apply to all the columns or different functions per column

In [17]:
student_data = pd.DataFrame({'Name':['Abel','Bale','Chris','Davies','Emma','Frank'],
                             'Score':[85,92,58,70,95,62], 
                             'Age':[17,19,16,22,25,20],
                             'Subject':['Math','Math','English','English','Math','English']})
student_data

,Name,Score,Age,Subject
0,Abel,85,17,Math
1,Bale,92,19,Math
2,Chris,58,16,English
3,Davies,70,22,English
4,Emma,95,25,Math
5,Frank,62,20,English


> **Suppose we grouped the dataset by subject and we wanted to compute the sum,min,max and mean for the Score column and the mean for the Age column**

In [18]:
grp_by_subject = student_data.groupby('Subject').agg({'Score':['sum','mean','min','max'],
                                                    'Age':'mean'})
grp_by_subject

Score                           Age
          sum       mean min max       mean
Subject                                    
English   190  63.333333  58  70  19.333333
Math      272  90.666667  85  95  20.333333

The resulting dataframe has hierarchical columns, the same you would get aggregating each column separately and using concat to glue the results. This was achieved by passing a dictionary into the agg function. 

### **2.2. Grouping with Custom-built Functions**
#### **This section treats the apply method**
Grouping with functions using Pandas is a powerful way to perform operations on subsets of data.
Apply splits the object beign manipulated into pieces, invokes the passed function on each piece, then concatenates the pieces together.


> **Grouping by a Custom Function**

First define a function that categorizes students based on their scores

In [19]:
def performance_category(score):
    if score > 75:
        return 'High'
    else:
        return 'Low'

> **Group by the Function**

In [20]:
student_data['Performance'] = student_data['Score'].apply(performance_category)
group_performance = student_data.groupby(student_data['Performance'])

> **Now retrieve the performers by iterating over the group**

In [21]:
for name , group in group_performance:
    print(f"\n{name} Performers:")
    print(group)


High Performers:
   Name  Score  Age Subject Performance
0  Abel     85   17    Math        High
1  Bale     92   19    Math        High
4  Emma     95   25    Math        High

Low Performers:
     Name  Score  Age  Subject Performance
2   Chris     58   16  English         Low
3  Davies     70   22  English         Low
5   Frank     62   20  English         Low


> **To have the result as a dataframe**

In [22]:
result_df = pd.concat([group for name, group in group_performance],axis=0)
result_df

,Name,Score,Age,Subject,Performance
0,Abel,85,17,Math,High
1,Bale,92,19,Math,High
4,Emma,95,25,Math,High
2,Chris,58,16,English,Low
3,Davies,70,22,English,Low
5,Frank,62,20,English,Low


<div style="background-color: black; padding: 5px">
    <p><b><font size="+2" color="white">3. Group-wise Operations & Transformations</font></b></p>
    </div>
    
Aggregation is only one kind of group operation. It is a special case in the more general class of data transformations; that is, it accepts functions that reduce a one-dimensional array to a scalar value.Transformations apply a function to each group but **return an object of the same shape as the input**, unlike aggregation , which **reduces the data size**.

### **This section introduces the transform and apply methods**
> #### **Given a dataframe that we want to add a column containing group means for each index**

One way to achieve this is to apply the function to each group and place the results in the appropriate locations. If each group produces a scalar value, it will be broadcasted



> #### **Given a Transformation Example**
Lets compute the average score in each subject

In [23]:
student_data['Average_Score'] = student_data.groupby('Performance')['Score'].transform(np.mean)
student_data

,Name,Score,Age,Subject,Performance,Average_Score
0,Abel,85,17,Math,High,90.666667
1,Bale,92,19,Math,High,90.666667
2,Chris,58,16,English,Low,63.333333
3,Davies,70,22,English,Low,63.333333
4,Emma,95,25,Math,High,90.666667
5,Frank,62,20,English,Low,63.333333


> **Suppose we want to subtract the mean value from each score. To do this we create a demeaning function**

In [24]:
def demean(Score):
    return Score - Score.mean()

demeaned = student_data.groupby('Performance')['Score'].transform(demean)
demeaned

0   -5.666667
1    1.333333
2   -5.333333
3    6.666667
4    4.333333
5   -1.333333
Name: Score, dtype: float64

<div style="background-color: black; padding: 5px">
    <p><b><font size="+2" color="white">4. Quantile & Binning Analysis</font></b></p>
    </div>
    
Pandas has tools for slicing data into bins - `cut` and `qcut`. Combining these with groupby allows one to perform quantile analysis on a dataset.

>#### **Going back to the titanic dataset a DataFrame**

In [25]:
dat.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S


In [26]:
factor = pd.cut(dat['Age'],4)
factor[:10]

0    (20.315, 40.21]
1    (20.315, 40.21]
2    (20.315, 40.21]
3    (20.315, 40.21]
4    (20.315, 40.21]
5                NaN
6    (40.21, 60.105]
7     (0.34, 20.315]
8    (20.315, 40.21]
9     (0.34, 20.315]
Name: Age, dtype: category
Categories (4, interval[float64, right]): [(0.34, 20.315] < (20.315, 40.21] < (40.21, 60.105] < (60.105, 80.0]]

The factor object returned by cut can be passed directly to groupby and this can be used to compute a set of statitics for another column.

In [27]:
def get_stats(group):
    return {'min':np.min(group), 'max':np.max(group),
            'count':group.count(),'mean':np.mean(group)}

grouped = dat['Fare'].groupby(factor)
grouped.apply(get_stats).unstack()

,min,max,count,mean
Age,,,,
"(0.34, 20.315]",0.0000,263.0000,179.0,29.853074
"(20.315, 40.21]",0.0000,512.3292,385.0,34.019327
"(40.21, 60.105]",0.0000,247.5208,128.0,42.348245
"(60.105, 80.0]",6.2375,263.0000,22.0,41.371214


This gave us equal age intervals. In order to get equal age intervals based on sample quantiles, we use qcut.

In [28]:
quantile_grps = pd.qcut(dat['Age'], 10, labels=False)
#Now group the fares according to the age groups
grouped = dat['Fare'].groupby(quantile_grps)
grouped.apply(get_stats).unstack()

,min,max,count,mean
Age,,,,
0.0,7.2292,151.5500,77.0,31.928464
1.0,0.0000,263.0000,87.0,31.676390
2.0,4.0125,262.3750,67.0,23.624128
3.0,0.0000,263.0000,70.0,34.826190
4.0,6.9750,211.5000,61.0,23.206075
5.0,7.0458,211.3375,66.0,28.009721
6.0,0.0000,512.3292,91.0,44.517718
7.0,0.0000,227.5250,53.0,40.532308
8.0,0.0000,247.5208,78.0,42.085260


<div style="background-color: black; padding: 5px">
    <p><b><font size="+2" color="white">Practical Case 1 - Filling Missing Values with Group values</font></b></p>
    </div>
    
When cleaning up missing data, most times you will filter out data observations using `dropna` but in some situations, you may want to fill in the NA values with a derived or fixed value. `Fillna` is the tool dedicated for this purpose.
> #### **Given the titanic dataset, lets fill in the missing age with the mean male age**

In [29]:
male_data = data[data['Sex']=='male']
data['filled_age_data'] = data['Age'].fillna(round(np.mean(male_data['Age']),1))
data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,filled_age_data
580,581,1,2,female,25.0,1,1,237789,30.0000,NaN,S,25.0
841,842,0,2,male,16.0,0,0,S.O./P.P. 3,10.5000,NaN,S,16.0
406,407,0,3,male,51.0,0,0,347064,7.7500,NaN,S,51.0
394,395,1,3,female,24.0,0,2,PP 9549,16.7000,G6,S,24.0
453,454,1,1,male,49.0,1,0,17453,89.1042,C92,C,49.0
671,672,0,1,male,31.0,1,0,F.C. 12750,52.0000,B71,S,31.0
257,258,1,1,female,30.0,0,0,110152,86.5000,B77,S,30.0
528,529,0,3,male,39.0,0,0,3101296,7.9250,NaN,S,39.0
433,434,0,3,male,17.0,0,0,STON/O 2. 3101274,7.1250,NaN,S,17.0
773,774,0,3,male,NaN,0,0,2674,7.2250,NaN,C,33.8


> #### **Another way is to fill with the group values using groupby**
**group the ages by sex and fill the missing age values with the mean value of the sex group**

In [30]:
data.groupby('Sex')['Age'].mean()

Sex
female    26.333333
male      33.833333
Name: Age, dtype: float64

> **Now fill the NA values using the group means**

In [31]:
fill_mean = lambda g: g.fillna(round(g.mean(),2))
data['filled_NA_Age'] = data.groupby('Sex')['Age'].apply(fill_mean)
data

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,filled_age_data,filled_NA_Age
580,581,1,2,female,25.0,1,1,237789,30.0000,NaN,S,25.0,25.00
841,842,0,2,male,16.0,0,0,S.O./P.P. 3,10.5000,NaN,S,16.0,16.00
406,407,0,3,male,51.0,0,0,347064,7.7500,NaN,S,51.0,51.00
394,395,1,3,female,24.0,0,2,PP 9549,16.7000,G6,S,24.0,24.00
453,454,1,1,male,49.0,1,0,17453,89.1042,C92,C,49.0,49.00
671,672,0,1,male,31.0,1,0,F.C. 12750,52.0000,B71,S,31.0,31.00
257,258,1,1,female,30.0,0,0,110152,86.5000,B77,S,30.0,30.00
528,529,0,3,male,39.0,0,0,3101296,7.9250,NaN,S,39.0,39.00
433,434,0,3,male,17.0,0,0,STON/O 2. 3101274,7.1250,NaN,S,17.0,17.00
773,774,0,3,male,NaN,0,0,2674,7.2250,NaN,C,33.8,33.83


<div style="background-color: black; padding: 5px">
    <p><b><font size="+2" color="white">Practical Case 2 - Group Weighted Average Calculations</font></b></p>
    </div>

Under the split-apply combine paradigm of groupby, operations between columns in a dataFrame or two Series such as group weighted average are a common affair.

> #### **Given a dataset containing group keys, values and some weights**

In [32]:
data2 = pd.DataFrame({'category':['A','A','A','A','B','B','B','B'],
                     'data': np.random.rand(8)*100,
                     'weights': np.random.rand(8)})
data2

,category,data,weights
0,A,39.563928,0.210273
1,A,80.463728,0.565665
2,A,53.413741,0.480708
3,A,67.117755,0.216354
4,B,82.020898,0.690929
5,B,10.903549,0.298858
6,B,95.537955,0.724278
7,B,59.485040,0.297207


**To now compute the group weighted average by category**

In [33]:
grp_data = data2.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grp_data.apply(get_wavg)

category
A    63.837310
B    72.990936
dtype: float64

<div style="background-color: black; padding: 5px">
    <p><b><font size="+2" color="white">Advanced Grouping - groupby using pd.Grouper</font></b></p>
    </div>
    
The pd.Grouper is useful for grouping time series data or other advanced groupings
> #### **Assuming we have a date column in the titanic dataset**

In [34]:
dat['Date'] = pd.date_range(start='1/1/1915',periods=len(dat),freq='D')
grouped_date = dat.groupby(pd.Grouper(key='Date',freq='Y')).agg({'Fare':'sum','Survived':'mean'})
grouped_date

,Fare,Survived
Date,,
1915-12-31,11795.8330,0.383562
1916-12-31,12096.2082,0.398907
1917-12-31,4801.9081,0.350000


<div style="background-color: black; padding: 5px">
    <p><b><font size="+2" color="white">Pivot Tables</font></b></p>
    </div>

A Pivot table is a data summarization tool that aggregates tabular data by one or more keys. This operation arranges the data in a rectangle with some of the group keys along the rows and some along the columns. In pandas, pivot table can be made using:
1. groupby combined with reshape operations using hierarchical indexing
2. dataframes pivot_table method
3. pandas.pivot_table function

Pivot_table can be used to add partial totals called margins

> #### **Given the tips dataset**

In [35]:
tips_data = pd.read_csv('../../../Data/tips.csv')
tips_data.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


> **Assumming we wanted to compute a table of group means  arranged by sex and smoker on the rows**

In [36]:
tips_pvtab = pd.pivot_table(tips_data,values=['tip','size','total_bill'],
                           index=['day','smoker'],aggfunc=np.mean)
tips_pvtab

size       tip  total_bill
day  smoker                                
Fri  No      2.250000  2.812500   18.420000
     Yes     2.066667  2.714000   16.813333
Sat  No      2.555556  3.102889   19.661778
     Yes     2.476190  2.875476   21.276667
Sun  No      2.929825  3.167895   20.506667
     Yes     2.578947  3.516842   24.120000
Thur No      2.488889  2.673778   17.113111
     Yes     2.352941  3.030000   19.190588